In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import seaborn as sns
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [2]:
df = pd.read_csv('/Users/priya/Downloads/train.csv')

In [3]:
target = df.pop('TARGET_5Yrs')

In [4]:
df.drop('Id_old', axis=1, inplace=True)
df.drop('Id', axis=1, inplace=True)

In [5]:
df.columns = df.columns.str.strip()
df

,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV
0,80,24.3,7.8,3.0,6.4,45.7,0.1,0.3,22.6,2.0,2.9,72.1,2.2,2.0,3.8,3.2,1.1,0.2,1.6
1,75,21.8,10.5,4.2,7.9,55.1,-0.3,-1.0,34.9,2.4,3.6,67.8,3.6,3.7,6.6,0.7,0.5,0.6,1.4
2,85,19.1,4.5,1.9,4.5,42.8,0.4,1.2,34.3,0.4,0.6,75.7,0.6,1.8,2.4,0.8,0.4,0.2,0.6
3,63,19.1,8.2,3.5,6.7,52.5,0.3,0.8,23.7,0.9,1.5,66.9,0.8,2.0,3.0,1.8,0.4,0.1,1.9
4,63,17.8,3.7,1.7,3.4,50.8,0.5,1.4,13.7,0.2,0.5,54.0,2.4,2.7,4.9,0.4,0.4,0.6,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,32,9.2,1.8,0.7,1.8,40.3,-0.1,-0.2,23.1,0.4,0.6,65.7,0.3,1.8,1.9,0.5,0.3,0.2,0.4
7996,54,6.0,1.8,0.7,1.4,48.7,0.1,0.1,3.1,0.2,0.4,70.1,1.0,1.1,2.0,0.1,0.0,0.3,0.3
7997,85,28.2,10.7,4.0,9.0,45.1,0.2,0.6,23.6,2.8,3.9,69.7,1.0,2.1,3.1,3.4,1.2,0.2,1.8
7998,39,7.7,2.5,1.0,2.3,40.1,-0.3,-0.5,13.3,0.6,0.7,74.3,0.4,0.6,0.9,0.2,0.3,0.3,0.5


In [6]:
#df_cleaned = df.drop(['FG%', '3P%', 'FT%'], axis = 1) 

In [7]:
from sklearn.svm import LinearSVC
lin_clf = LinearSVC(random_state=42)

X_train, X_test, y_train, y_test = train_test_split(df, target, test_size=0.2, random_state=8)

lin_clf.fit(X_train, y_train)

from sklearn.metrics import accuracy_score

y_pred = lin_clf.predict(X_test)

accuracy_score(y_test, y_pred)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.839375

In [8]:
#scaler = StandardScaler()
#df_cleaned = scaler.fit_transform(df_cleaned)

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train.astype(np.float32))

X_test_scaled = scaler.transform(X_test.astype(np.float32))

lin_clf = LinearSVC(random_state=42)

lin_clf.fit(X_train_scaled, y_train)

y_pred = lin_clf.predict(X_test_scaled)

accuracy_score(y_test, y_pred)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.8375

In [10]:
from sklearn.svm import SVC

svm_clf = SVC(gamma= "scale", probability = True)

svm_clf.fit(X_train_scaled, y_train) # We use an SVC with an RBF kernel

y_pred = svm_clf.predict(X_test_scaled)

accuracy_score(y_test, y_pred)

0.839375

In [11]:
lin_clf = LinearSVC(random_state=42)

In [12]:
from sklearn.model_selection import RandomizedSearchCV

from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}

rnd_search_cv = RandomizedSearchCV(svm_clf, param_distributions, n_iter=10, verbose=2, cv=3)

rnd_search_cv.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] C=10.115423518066585, gamma=0.010232773999779603 ................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] . C=10.115423518066585, gamma=0.010232773999779603, total=   8.7s
[CV] C=10.115423518066585, gamma=0.010232773999779603 ................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.7s remaining:    0.0s


[CV] . C=10.115423518066585, gamma=0.010232773999779603, total=   7.7s
[CV] C=10.115423518066585, gamma=0.010232773999779603 ................
[CV] . C=10.115423518066585, gamma=0.010232773999779603, total=   8.1s
[CV] C=3.1839910100445934, gamma=0.07727254844914681 .................
[CV] .. C=3.1839910100445934, gamma=0.07727254844914681, total=   3.8s
[CV] C=3.1839910100445934, gamma=0.07727254844914681 .................
[CV] .. C=3.1839910100445934, gamma=0.07727254844914681, total=   3.7s
[CV] C=3.1839910100445934, gamma=0.07727254844914681 .................
[CV] .. C=3.1839910100445934, gamma=0.07727254844914681, total=   3.5s
[CV] C=9.802323045188825, gamma=0.03909274119068494 ..................
[CV] ... C=9.802323045188825, gamma=0.03909274119068494, total=   5.2s
[CV] C=9.802323045188825, gamma=0.03909274119068494 ..................
[CV] ... C=9.802323045188825, gamma=0.03909274119068494, total=   5.1s
[CV] C=9.802323045188825, gamma=0.03909274119068494 ..................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  2.9min finished


RandomizedSearchCV(cv=3, estimator=SVC(probability=True),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa1031be5d0>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa1031be2d0>},
                   verbose=2)

In [13]:
rnd_search_cv.best_estimator_

SVC(C=10.805992119429055, gamma=0.0011410756190133008, probability=True)

In [14]:
rnd_search_cv.best_score_

0.8326561685994457

In [15]:
rnd_search_cv.best_estimator_.fit(X_train_scaled, y_train)

y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
print(accuracy_score(y_train, y_pred))
y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.8325


0.83875

In [16]:
rnd_search_cv.best_estimator_.predict_proba(X_test)
#clf.predict_proba(X_test)



array([[0.32640937, 0.67359063],
       [0.32632284, 0.67367716],
       [0.32642152, 0.67357848],
       ...,
       [0.32642095, 0.67357905],
       [0.32635388, 0.67364612],
       [0.32642252, 0.67357748]])

In [17]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[   0  257]
 [   1 1342]]


In [18]:
#from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
#print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [19]:

#print("Precision:",metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
#print("Recall:",metrics.recall_score(y_test, y_pred))

In [20]:
#clf.score(X_test, y_test)
#clf.predict_proba(X_test)

In [21]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[   0  257]
 [   1 1342]]


In [22]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       257
           1       0.84      1.00      0.91      1343

    accuracy                           0.84      1600
   macro avg       0.42      0.50      0.46      1600
weighted avg       0.70      0.84      0.77      1600



In [30]:
roc=roc_auc_score(y_test, rnd_search_cv.best_estimator_.predict_proba(X_test_scaled)[:,1])
roc

0.6066243470249253

In [17]:
#from sklearn.model_selection import GridSearchCV 
#from sklearn.svm import SVC 
  
# defining parameter range 
#param_grid = {'C': [0.1, 1, 10, 100, 1000],  
 #             'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
  #            'kernel': ['rbf']}  
  
#grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
#grid.fit(X_train, y_train) 

In [18]:
# print best parameter after tuning 
#print(grid.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
#print(grid.best_estimator_) 

In [19]:
#grid_predictions = grid.predict(X_test) 
  
# print classification report 
#print(classification_report(y_test, grid_predictions)) 

In [20]:
#roc=roc_auc_score(y_test, xgb_cv.predict_proba(X_test)[:,1])
#roc


In [24]:
df1 = pd.read_csv('/Users/priya/Downloads/test.csv')
df1.drop('Id', axis=1, inplace=True)
df1.drop('Id_old', axis=1, inplace=True)

In [25]:
#df2 = df1.drop(['FG%', '3P%', 'FT%'], axis = 1) 

df_test_scaled = scaler.transform(df1.astype(np.float32))

In [28]:
final = rnd_search_cv.best_estimator_.predict_proba(df_test_scaled)[:,1]

In [29]:
final

array([0.8327257 , 0.83264092, 0.83275107, ..., 0.83266146, 0.83278938,
       0.83274454])

In [31]:
pd.DataFrame(final).to_csv("/Users/priya/Downloads/13684182_week2_svm_hpt.csv")